### Definição de funções e imports necessários

In [1]:
import pandas as pd
import numpy as np
import json
import excel2json
from geopy.geocoders import Nominatim
import pymongo

**Funções:**

In [49]:
def readCSV (path):
    return pd.read_csv(path, sep = ";");

def readXLSX (path):
    return pd.read_excel(path, index_col=None);

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORD>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def dataframeToJSONConverter (dataframe):
    dataJSON = dataframe.to_json(orient="records")
    return json.loads(dataJSON)

def deletingColumns (dataframe):
    columns = ["Região", 
               "UF", 
               "Código do Município", 
               "Nome do Município", 
               "Nome da Escola", 
               "Localização", 
               "Dependência Administrativa"];
    
    for column in columns:
        try:
            dataframe.pop(column)
        except:
            continue
    
def JSONstructure (data):    
    dataJSON = {"Média Horas-Aula diárias": 
                {
                "Última atualização": data["Ano"],
                "Educação Infantil": 
                    {
                        "Total": data["TotalI"],
                        "Creche": data["Creche"],
                        "Pré-Escola": data["Pré-Escola"]
                    },
                "Ensino Fundamental 8 e 9 anos": 
                    {
                        "Total": data["TotalF"],
                        "Anos Iniciais": data["Anos Iniciais"],
                        "Anos Finais": data["Anos Finais"],
                        "1º Ano": data["1° ano"],
                        "2º Ano": data["2° ano"],
                        "3º Ano": data["3° ano"],
                        "4º Ano": data["4° ano"],
                        "5º Ano": data["5° ano"],
                        "6º Ano": data["6° ano"],
                        "7º Ano": data["7° ano"],
                        "8º Ano": data["8° ano"],
                        "9º Ano": data["9° ano"],
                        #"Turmas Multietapa, Multi ou Correção de Fluxo 2": data["Turmas Multietapa, Multi ou Correção de Fluxo 2"]
                    },
                "Ensino Médio": 
                    {
                        "Total": data["TotalM"],
                        "1ª série": data["1ª série"],
                        "2ª série": data["2ª série"],
                        "3ª série": data["3ª série"],
                        "4ª série": data["4ª série"],
                        "Não-Seriado": data["Não-Seriado"]
                    }
        }};

    return dataJSON;

def filterPelotas (dataframe):
    filter = dataframe["Nome do Município"] == "Pelotas"
    
    filterPelotas = dataframe[filter]
    filterPelotas.reset_index(drop=True, inplace=True)
    
    return filterPelotas

def insertIntoDB (dataframe, collection):
    for index, row in dataframe.iterrows():
        teste1 = JSONstructure(row);

        collection.update_one({"Código INEP": row["Código da Escola"]}, {"$set": teste1});

**Conexão com o BD:**

In [35]:
collection = openDataBaseMongoDB()

### ETL: Informações das Escolas

In [9]:
infoSchools = readCSV(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\InformacoesGeraisDasEscolas.csv');
infoSchools.head()

,Restrição de Atendimento,Escola,Código INEP,UF,Município,Localização,Localidade Diferenciada,Categoria Administrativa,Endereço,Telefone,Dependência Administrativa,Categoria Escola Privada,Conveniada Poder Público,Regulamentação pelo Conselho de Educação,Porte da Escola,Etapas e Modalidade de Ensino Oferecidas,Outras Ofertas Educacionais,Latitude,Longitude
0,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,COLEGIO TIRADENTES PELOTAS,43000304,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Pública,"AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003...",(53) 32812866,Estadual,Não Informado,Não,Sim,Entre 201 e 500 matrículas de escolarização,Ensino Médio,NaN,-31.75701,-52.37308
1,ESCOLA PARALISADA,EEI NOSSOS AMIGUINHOS,43001084,RS,Pelotas,Urbana,Não Informado,Privada,"RUA VOLUNTARIOS DA PATRIA, 1927. CENTRO. 96015...",(53) 30256230,Privada,Não Informado,Não,NaN,NaN,NaN,NaN,NaN,NaN
2,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CRIANCA AMADA,43001165,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. ...",(53) 30282009,Privada,Particular,Não,Sim,Entre 51 e 200 matrículas de escolarização,Educação Infantil,NaN,NaN,NaN
3,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI MUNDO DO MICKEY,43001173,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BALDOMERO TRAPAGA, 132 PORTO. 96075-540 Pe...",(53) 32279908,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.77139,-52.32994
4,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CHAPEUZINHO VERMELHO,43002145,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BARAO DE SANTA TECLA, 621 CENTRO. 96010-14...",(53) 81265782,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.76221,-52.34358


In [10]:
infoSchools.shape

(302, 19)

In [11]:
infoSchools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 19 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Restrição de Atendimento                  302 non-null    object 
 1   Escola                                    302 non-null    object 
 2   Código INEP                               302 non-null    int64  
 3   UF                                        302 non-null    object 
 4   Município                                 302 non-null    object 
 5   Localização                               302 non-null    object 
 6   Localidade Diferenciada                   302 non-null    object 
 7   Categoria Administrativa                  302 non-null    object 
 8   Endereço                                  302 non-null    object 
 9   Telefone                                  292 non-null    object 
 10  Dependência Administrativa            

In [12]:
infoSchools.isnull().sum()

Restrição de Atendimento                      0
Escola                                        0
Código INEP                                   0
UF                                            0
Município                                     0
Localização                                   0
Localidade Diferenciada                       0
Categoria Administrativa                      0
Endereço                                      0
Telefone                                     10
Dependência Administrativa                    0
Categoria Escola Privada                      0
Conveniada Poder Público                      0
Regulamentação pelo Conselho de Educação     63
Porte da Escola                              63
Etapas e Modalidade de Ensino Oferecidas     66
Outras Ofertas Educacionais                 212
Latitude                                    104
Longitude                                   104
dtype: int64

**Processo de Geocoding:**

In [43]:
infoSchools["Endereço"][2]

'AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. 96085-470 Pelotas - RS.'

In [82]:
geolocator = Nominatim(user_agent = "ScoolarData")
latitude = []
longitude = []

for local in collectionINEP["Endereço"]:
    try:
        location = geolocator.geocode(local)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        continue

**Conversão do *dataframe* para json:**

In [13]:
schoolData = dataframeToJSONConverter (infoSchools)
schoolData[0]

{'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308}

### ETL: Média de estudates por escola

In [53]:
studentsAVG = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\MediaDeAlunosPorTurma.xlsx')

In [54]:
studentsAVG.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,--,...,--,--,--,7,--,--,--,--,--,--
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,...,--,--,--,--,--,--,--,--,--,--
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,--,...,--,--,--,10,--,--,--,--,--,--
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,10,...,7,15,8,23.5,--,--,--,--,--,--
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,16,...,20,26,18,--,--,--,--,--,--,--


In [55]:
studentsAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176386 entries, 0 to 176385
Data columns (total 31 columns):
 #   Column                                           Non-Null Count   Dtype 
---  ------                                           --------------   ----- 
 0   Ano                                              176386 non-null  int64 
 1   Região                                           176386 non-null  object
 2   UF                                               176386 non-null  object
 3   Código do Município                              176386 non-null  int64 
 4   Nome do Município                                176386 non-null  object
 5   Código da Escola                                 176386 non-null  int64 
 6   Nome da Escola                                   176386 non-null  object
 7   Localização                                      176386 non-null  object
 8   Dependência Administrativa                       176386 non-null  object
 9   TotalI                    

In [56]:
studentsAVG_Pelotas = filterPelotas(studentsAVG)
studentsAVG_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,26.6,25,26,30,--,--
1,2019,Sul,RS,4314407,Pelotas,43001165,EEI CRIANCA AMADA,Urbana,Privada,14,...,--,--,--,--,--,--,--,--,--,--
2,2019,Sul,RS,4314407,Pelotas,43001173,EEI MUNDO DO MICKEY,Urbana,Privada,3,...,--,--,--,--,--,--,--,--,--,--
3,2019,Sul,RS,4314407,Pelotas,43002145,EEI CHAPEUZINHO VERMELHO,Urbana,Privada,3.5,...,--,--,--,--,--,--,--,--,--,--
4,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,--,...,30.5,37,--,--,--,--,--,--,--,--


In [57]:
studentsAVG_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 31 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Ano                                              228 non-null    int64 
 1   Região                                           228 non-null    object
 2   UF                                               228 non-null    object
 3   Código do Município                              228 non-null    int64 
 4   Nome do Município                                228 non-null    object
 5   Código da Escola                                 228 non-null    int64 
 6   Nome da Escola                                   228 non-null    object
 7   Localização                                      228 non-null    object
 8   Dependência Administrativa                       228 non-null    object
 9   TotalI                                     

In [58]:
deletingColumns(studentsAVG_Pelotas)

In [59]:
studentsAVG_Pelotas.head()

,Ano,Código da Escola,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,43000304,--,--,--,--,--,--,--,--,...,--,--,--,--,26.6,25,26,30,--,--
1,2019,43001165,14,13.6,15,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2,2019,43001173,3,6,1.5,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
3,2019,43002145,3.5,4.3,2.7,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
4,2019,43003222,--,--,--,29.4,27.3,34.7,26.7,27.7,...,30.5,37,--,--,--,--,--,--,--,--


**Inserindo no BD:**

In [46]:
insertIntoDB(studentsAVG_Pelotas, collection)

In [47]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f22117d86818b0338731eb4'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Média de Alunos por Turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'

### ETL: Horas-aula diária por escola

In [6]:
hoursAVG = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\MediaHorasAulaDiariaPorEscola.xlsx')

In [7]:
hoursAVG.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,NaN,...,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024828,EMEIEF IZIDORO STEDILE,Urbana,Municipal,4.0,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024917,EMEIEF JOSE BASILIO DA GAMA,Rural,Municipal,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
hoursAVG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152509 entries, 0 to 152508
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Ano                         152509 non-null  int64  
 1   Região                      152509 non-null  object 
 2   UF                          152509 non-null  object 
 3   Código do Município         152509 non-null  int64  
 4   Nome do Município           152509 non-null  object 
 5   Código da Escola            152509 non-null  int64  
 6   Nome da Escola              152509 non-null  object 
 7   Localização                 152509 non-null  object 
 8   Dependência Administrativa  152509 non-null  object 
 9   TotalI                      94830 non-null   float64
 10  Creche                      59884 non-null   float64
 11  Pré-Escola                  82102 non-null   float64
 12  TotalF                      98340 non-null   float64
 13  Anos Iniciais 

In [52]:
hoursAVG_Pelotas = filterPelotas(hoursAVG)
hoursAVG_Pelotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Ano                         224 non-null    int64  
 1   Região                      224 non-null    object 
 2   UF                          224 non-null    object 
 3   Código do Município         224 non-null    int64  
 4   Nome do Município           224 non-null    object 
 5   Código da Escola            224 non-null    int64  
 6   Nome da Escola              224 non-null    object 
 7   Localização                 224 non-null    object 
 8   Dependência Administrativa  224 non-null    object 
 9   TotalI                      161 non-null    float64
 10  Creche                      95 non-null     float64
 11  Pré-Escola                  157 non-null    float64
 12  TotalF                      119 non-null    float64
 13  Anos Iniciais               112 non

In [15]:
deletingColumns(hoursAVG_Pelotas)

In [17]:
hoursAVG_Pelotas.head()

,Ano,Código da Escola,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,...,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2019,43000304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.9,8.9,8.9,8.9,NaN,NaN
1,2019,43001165,12.0,12.0,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,43001173,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,43002145,5.5,5.4,5.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,43003222,NaN,NaN,NaN,4.2,4.3,4.1,4.3,4.3,...,4.1,4.1,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Inserindo no BD:**

In [50]:
insertIntoDB(hoursAVG_Pelotas, collection)

In [51]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f22117d86818b0338731eb4'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Média de Alunos por Turma': {'Última atualização': 2019,
  'Educação Infantil': {'Total': '--', 'Creche': '--'